# Stock Visualization
The goal of the notebook is to take a systematic approach to being able take company's stock name and utilizing the Alpha Vantage api to create a series of visualizations and critiques based on the information provided.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import json
import matplotlib.dates as mdates
import warnings
from pandas.io.json import json_normalize
warnings.filterwarnings('ignore')

from APIData import getAPIKey

We're going to start doing some api calls. For the time being i'm listing out the variables needed to make the api calls. Eventually, I'll put it into a class object for easier access

In [2]:
apiKey = getAPIKey()
symbol = 'OSTK'
type = 'TIME_SERIES_DAILY'
dataType = 'csv'
intervalInMinutes = 5
url = f'https://www.alphavantage.co/query?function={type}&symbol={symbol}&apikey={apiKey}'

intraDayURL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={intervalInMinutes}min&outputsize=full&apikey={apiKey}'
print('Intra Day ')
csvDownload = f'https://www.alphavantage.co/query?function={type}&symbol={symbol}&apikey={apiKey}&datatype={dataType}'
intraDayCSV = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={apiKey}&outputsize=full&datatype=csv'
#print('Download CSV:', csvDownload)
print(intraDayCSV)
exit(0)
with urllib.request.urlopen(intraDayURL) as response:
   html = response.read()
   data = json.loads(html)
print(data)

stockData = []
#stockInfo = pd.DataFrame(data['Time Series (Daily)'])
stockInfo = stockInfo.transpose()

Intra Day 
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=OSTK&interval=1min&apikey=YZ4TPATE6WB2OHPH&outputsize=full&datatype=csv
{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume', '2. Symbol': 'OSTK', '3. Last Refreshed': '2019-06-21 16:00:00', '4. Interval': '5min', '5. Output Size': 'Full size', '6. Time Zone': 'US/Eastern'}, 'Time Series (5min)': {'2019-06-21 16:00:00': {'1. open': '11.5200', '2. high': '11.6000', '3. low': '11.4800', '4. close': '11.6000', '5. volume': '410840'}, '2019-06-21 15:55:00': {'1. open': '11.5600', '2. high': '11.6200', '3. low': '11.4900', '4. close': '11.5175', '5. volume': '227451'}, '2019-06-21 15:50:00': {'1. open': '11.6831', '2. high': '11.7000', '3. low': '11.5600', '4. close': '11.5600', '5. volume': '126906'}, '2019-06-21 15:45:00': {'1. open': '11.5700', '2. high': '11.6800', '3. low': '11.4900', '4. close': '11.6800', '5. volume': '200671'}, '2019-06-21 15:40:00': {'1. open': '

NameError: name 'stockInfo' is not defined

In [ ]:
stockInfo.head()

In [ ]:
stockInfo.shape

In [ ]:
stockInfo.columns

In [ ]:
stockInfo = stockInfo.rename(columns = {'1. open': 'Open',
                            '2. high': 'High',
                            '3. low': 'Low',
                            '4. close': 'Close',
                            '5. volume': 'Volume'})
stockInfo['Date'] = stockInfo.index
stockInfo.sort_values(by=['Date'], inplace=True)
stockInfo.head()

In [ ]:
stockInfo.info()

The dataypes are being listed as string objects. They need to be adjusted so that we can perform the calculations correctly.

In [ ]:
stockInfo.Open = pd.to_numeric(stockInfo.Open)
stockInfo.High = pd.to_numeric(stockInfo.High)
stockInfo.Low = pd.to_numeric(stockInfo.Low)
stockInfo.Close = pd.to_numeric(stockInfo.Close)
stockInfo.Volume = pd.to_numeric(stockInfo.Volume)
stockInfo.Date = pd.to_datetime(stockInfo.Date)
stockInfo.info()

We can see that we have 100 rows with 5 columns each. I'm going to rename the columns to make it a bit easier to call later on

In [ ]:
plt.figure(figsize = (12, 8))
ax = plt.subplot()
plt.plot(stockInfo.Date, stockInfo.Close, label = 'Close Price', color = 'blue')
#plt.plot(stockInfo.Date, stockInfo.Open, label = 'Open Price', color = 'green')
#plt.plot(stockInfo.Date, stockInfo.High, label = 'High', color = 'red')
#plt.plot(stockInfo.Date, stockInfo.Low, label = 'Low', color = 'orange')
plt.title(symbol)
plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.locator_params(axis='x', numticks=3)
plt.legend()
plt.gcf().autofmt_xdate()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.show()

Now that we have a basic graph drawn, I want to draw out a stochastic oscillator for the stock.
A Stochastic Oscillator is a momentum indicator comparing a particular closing price of a security to a range of its prices over a certain period of time. The sensitivity of the oscillator to market movements is reducible by adjusting that time period or by taking a moving average of the result.
The equation to calculate the stochastic oscillator is:

%K = 100(C – L14)/(H14 – L14)

Where:

C = the most recent closing price

L14 = the low of the 14 previous trading sessions

H14 = the highest price traded during the same 14-day period

%K= the current market rate for the currency pair

%D = 3-period moving average of %K


In [ ]:
#Create the "L14" column in the DataFrame
stockInfo['L14'] = stockInfo['Low'].rolling(window=14).min()
#Create the "H14" column in the DataFrame
stockInfo['H14'] = stockInfo['High'].rolling(window=14).max()
#Create the "%K" column in the DataFrame
stockInfo['%K'] = 100*((stockInfo['Close'] - stockInfo['L14']) / (stockInfo['H14'] - stockInfo['L14']) )
#Create the "%D" column in the DataFrame
stockInfo['%D'] = stockInfo['%K'].rolling(window=3).mean()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1,figsize=(20,10))
stockInfo['Close'].plot(ax=axes[0]); axes[0].set_title(symbol + ' Close Prices')
stockInfo[['%K','%D']].plot(ax=axes[1]); axes[1].set_title(symbol + ' Stochastic Oscillator')
plt.hlines(80, 0, 100, linestyles = 'dashed', color = 'red', data = stockInfo.Date)
plt.hlines(20, 0, 100, linestyles = 'dashed', color = 'red', data = stockInfo.Date)
plt.hlines(50, 0, 100, linestyles = 'dashed', color = 'black', data = stockInfo.Date)

In [ ]:
stockInfo['Sell Entry'] = ((stockInfo['%K'] < stockInfo['%D']) & (stockInfo['%K'].shift(1) > stockInfo['%D'].shift(1))) & (stockInfo['%D'] > 80) 
stockInfo['Sell Exit'] = ((stockInfo['%K'] > stockInfo['%D']) & (stockInfo['%K'].shift(1) < stockInfo['%D'].shift(1))) 
stockInfo['Short'] = np.nan 
stockInfo.loc[stockInfo['Sell Entry'],'Short'] = -1 
stockInfo.loc[stockInfo['Sell Exit'],'Short'] = 0 
stockInfo['Short'][0] = 0 
stockInfo['Short'] = stockInfo['Short'].fillna(method='pad') 
stockInfo['Buy Entry'] = ((stockInfo['%K'] > stockInfo['%D']) & (stockInfo['%K'].shift(1) < stockInfo['%D'].shift(1))) & (stockInfo['%D'] < 20) 
stockInfo['Buy Exit'] = ((stockInfo['%K'] < stockInfo['%D']) & (stockInfo['%K'].shift(1) > stockInfo['%D'].shift(1))) 
stockInfo['Long'] = np.nan  
stockInfo.loc[stockInfo['Buy Entry'],'Long'] = 1  
stockInfo.loc[stockInfo['Buy Exit'],'Long'] = 0  
stockInfo['Long'][0] = 0  
stockInfo['Long'] = stockInfo['Long'].fillna(method='pad') 

#Add Long and Short positions together to get final strategy position (1 for long, -1 for short and 0 for flat) 
stockInfo['Position'] = stockInfo['Long'] + stockInfo['Short']
stockInfo['Position'].plot(figsize=(20,10))